In [1]:
import numpy as np
import os
print(os.getcwd())

dim0 = 64

in1 = np.random.rand(dim0, dim0).astype(np.float32)
in2 = np.random.rand(dim0, dim0).astype(np.float32)

print("in1:", in1)
print("in2:", in2)

/root/jupyter_notebooks/getting_started/matmul_optimized_64
in1: [[0.18551987 0.55881405 0.06903154 ... 0.7836606  0.9759704  0.10005483]
 [0.58072215 0.36215404 0.7735819  ... 0.81800324 0.4928764  0.9667422 ]
 [0.40606168 0.47879776 0.78154486 ... 0.66722256 0.11370797 0.47025064]
 ...
 [0.16393428 0.7999948  0.30212393 ... 0.39355478 0.91294557 0.9175907 ]
 [0.5870255  0.31948033 0.34665135 ... 0.5582657  0.0195512  0.79026246]
 [0.9154149  0.03640583 0.75276977 ... 0.47996557 0.63580406 0.45229474]]
in2: [[0.21619132 0.37853774 0.21801326 ... 0.07823552 0.6125641  0.38960606]
 [0.75060827 0.60506856 0.25092092 ... 0.5216007  0.28151667 0.3474155 ]
 [0.81571215 0.68858653 0.68913645 ... 0.0862569  0.3664147  0.2412752 ]
 ...
 [0.10239057 0.17770335 0.7781291  ... 0.3490614  0.3433643  0.29330888]
 [0.88543504 0.46614552 0.46715465 ... 0.6482386  0.05997119 0.62887746]
 [0.29001752 0.49429137 0.882026   ... 0.80744755 0.25104433 0.12207554]]


In [2]:
in1_py = np.resize(in1, (64,64))
in2_py = np.resize(in2, (64,64))

In [3]:
import time
start_time = time.time()
out_py = np.dot(in1_py,in2_py)
end_time = time.time()
python_time = end_time - start_time
print("SW matrix mul exe time: {}s".format(end_time - start_time))

SW matrix mul exe time: 0.0016901493072509766s


In [4]:
print("the result of software matmul: ", out_py)

the result of software matmul:  [[16.186634 15.095465 16.154686 ... 16.354357 14.952542 16.06986 ]
 [16.51513  17.769445 15.605106 ... 18.478514 15.45704  17.298456]
 [14.050013 15.466271 13.70028  ... 13.68868  13.435121 15.224033]
 ...
 [18.163101 17.869076 16.599476 ... 17.647184 15.95337  16.33484 ]
 [16.968243 17.295376 16.981222 ... 17.852009 14.485349 16.339268]
 [13.707981 15.05404  13.903966 ... 14.283077 14.542317 13.730874]]


In [5]:
from pynq import Overlay
overlay_baseline = Overlay("matmult_optimized.bit")
baseline = overlay_baseline.matmul_optimized_0

In [6]:
from pynq import allocate
input1_buffer0 = allocate(shape=(dim0*dim0,), dtype='f4')
input2_buffer0 = allocate(shape=(dim0*dim0,), dtype='f4')
output_buffer0 = allocate(shape=(dim0*dim0,), dtype='f4')

In [7]:
np.copyto(input1_buffer0, in1.flatten())
np.copyto(input2_buffer0, in2.flatten())

print("input1_buffer0:", input1_buffer0)
print("input2_buffer0:", input2_buffer0)
print("output_buffer0:", output_buffer0)

input1_buffer0: [0.18551987 0.55881405 0.06903154 ... 0.47996557 0.63580406 0.45229474]
input2_buffer0: [0.21619132 0.37853774 0.21801326 ... 0.80744755 0.25104433 0.12207554]
output_buffer0: [0. 0. 0. ... 0. 0. 0.]


In [8]:
input1_buffer0.sync_to_device()
input2_buffer0.sync_to_device()

print("input1_buffer0:", input1_buffer0)
print("input2_buffer0:", input2_buffer0)

input1_buffer0: [0.18551987 0.55881405 0.06903154 ... 0.47996557 0.63580406 0.45229474]
input2_buffer0: [0.21619132 0.37853774 0.21801326 ... 0.80744755 0.25104433 0.12207554]


In [9]:
baseline.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  A_1 = Register(A=write-only),
  A_2 = Register(A=write-only),
  B_1 = Register(B=write-only),
  B_2 = Register(B=write-only),
  Out_r_1 = Register(Out_r=write-only),
  Out_r_2 = Register(Out_r=write-only)
}

In [10]:
baseline.register_map.B_1.address

28

In [12]:
# Write A address (low and high)
baseline.write(baseline.register_map.A_1.address, input1_buffer0.physical_address & 0xFFFFFFFF)
baseline.write(baseline.register_map.A_2.address, (input1_buffer0.physical_address >> 32) & 0xFFFFFFFF)

# Write B address (low and high)
baseline.write(baseline.register_map.B_1.address, input2_buffer0.physical_address & 0xFFFFFFFF)
baseline.write(baseline.register_map.B_2.address, (input2_buffer0.physical_address >> 32) & 0xFFFFFFFF)

# Write AB address (low and high)
baseline.write(baseline.register_map.Out_r_1.address, output_buffer0.physical_address & 0xFFFFFFFF)
baseline.write(baseline.register_map.Out_r_2.address, (output_buffer0.physical_address >> 32) & 0xFFFFFFFF)

In [13]:
input1_buffer0.sync_to_device()
input2_buffer0.sync_to_device()

# Start the IP
baseline.write(0x00, 0x01)

# Wait for AP_DONE
while (baseline.read(0x00) & 0x2) == 0:
    pass

output_buffer0.sync_from_device()

In [14]:
baseline.write(0x00, 0x01)
start_time = time.time()
while True:
    reg = baseline.read(0x00)
    if reg != 1:
        break
end_time = time.time()
baseline_time = end_time - start_time
print("HW mul (baseline) exe time: {}s".format(end_time - start_time))

HW mul (baseline) exe time: 0.0004093647003173828s


In [15]:
output_buffer0.sync_from_device()
print("output_buffer0:", output_buffer0)

output_buffer0: [3.2824864 4.1621275 4.045168  ... 0.        0.        0.       ]


In [16]:
output_buffer0

PynqBuffer([3.2824864, 4.1621275, 4.045168 , ..., 0.       , 0.       ,
            0.       ], dtype=float32)

In [17]:
out_hw = output_buffer0.reshape((dim0, dim0))

print("Hardware result:\n", out_hw)
print("Software result:\n", out_py)

# Compare results
diff = np.abs(out_hw - out_py)
print("Max difference between HW and SW:", np.max(diff))

Hardware result:
 [[3.2824864 4.1621275 4.045168  ... 3.3981855 3.2467558 3.3062618]
 [3.2309437 3.59007   4.467357  ... 3.776995  4.1853304 5.070635 ]
 [2.836636  3.5225906 3.3423905 ... 3.1605554 3.3181562 3.9591773]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]
Software result:
 [[16.186634 15.095465 16.154686 ... 16.354357 14.952542 16.06986 ]
 [16.51513  17.769445 15.605106 ... 18.478514 15.45704  17.298456]
 [14.050013 15.466271 13.70028  ... 13.68868  13.435121 15.224033]
 ...
 [18.163101 17.869076 16.599476 ... 17.647184 15.95337  16.33484 ]
 [16.968243 17.295376 16.981222 ... 17.852009 14.485349 16.339268]
 [13.707981 15.05404  13.903966 ... 14.283077 14.542317 13.730874]]
Max difference between HW and SW: 20.978344
